In [2]:
import pandas as pd
import ast
import json
import re
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import warnings
from scipy.stats import pearsonr
warnings.filterwarnings('ignore')
from itertools import combinations
import random
from seqeval.metrics import f1_score,classification_report,accuracy_score
from sklearn.metrics import f1_score as bi_f1_score
from sklearn.metrics import accuracy_score as bi_accuracy_score
from sklearn.metrics import classification_report as bi_classification_report
from sklearn.metrics import roc_auc_score
import traceback
import html #for unescape & < >
from scipy import stats
import matplotlib.pyplot as plt
from collections import defaultdict
import emoji
import os
from nervaluate import Evaluator

In [842]:
data_200=pd.read_json('filtered_initial_batch.json')
data_3000=pd.read_csv('filtered_main_batches.json')
# data=pd.read_csv('/Users/ruofanhu/Desktop/Projects/USDA/Mturk/200_results_drop1.csv')
# data=pd.concat([data_200,d_3000])
dj_3000=data_3000.to_json(orient="records")
parsed_3000 = json.loads(dj_3000)

dj_200=data_200.to_json(orient="records")
parsed_200 = json.loads(dj_200)


In [843]:

def process_entity_label_200(record):
    txt = html.unescape(record['Input.tweet'])
    
    temp = list(filter(None, re.split('([,.!?:()[\]\\/"“”\s+])', txt)))

    # remove space strings from list and convert into np array
    tweet_split = np.array(list(filter(str.strip, temp)))


    token_labels = np.array(['O']*len(tweet_split),dtype=np.dtype(('U',10)))
#     if record['Answer.related_index'] != '[]' :
#         relation_lables_idx_str = sum([i.split(' ') for i in ast.literal_eval(record['Answer.related_index'])],[])

#         relation_lables_idx = list(map(int, relation_lables_idx_str))
#         relation_lables[relation_lables_idx] = 1

                
    if record['Answer.no-entity'] is None and re.split('[|]', record["Answer.html_output"])[1]!='': # the value is 1 when there is no entity to label
        html_output_list = ast.literal_eval(re.split('[|]', record["Answer.html_output"])[1])
                
        for e in html_output_list:
            if 'idx' in list(e.keys()):

                if ' ' in e['idx']:
                    idx = list(map(int, e['idx'].split(' ')))
                else:
                    idx = ast.literal_eval(e['idx'])

                if type(idx) is int:

#                             assert tweet_split[idx] == e['text']
                    token_labels[idx] = 'B-'+e['className'].split('-')[1]
                else:
                    idx=list(idx)
c
    return token_labels.tolist()

def process_entity_label_3000(record):
    txt = html.unescape(record['Input.tweet'])
    
    temp = list(filter(None, re.split('([,.!?:()[\]"\s+])', txt)))

    # remove space strings from list and convert into np array
    tweet_split = np.array(list(filter(str.strip, temp)))


    token_labels = np.array(['O']*len(tweet_split),dtype=np.dtype(('U',10)))
#     if record['Answer.related_index'] != '[]' :
#         relation_lables_idx_str = sum([i.split(' ') for i in ast.literal_eval(record['Answer.related_index'])],[])

#         relation_lables_idx = list(map(int, relation_lables_idx_str))
#         relation_lables[relation_lables_idx] = 1

                
    if record['Answer.no-entity'] is None and re.split('[|]', record["Answer.html_output"])[1]!='': # the value is 1 when there is no entity to label
        html_output_list = ast.literal_eval(re.split('[|]', record["Answer.html_output"])[1])
                
        for e in html_output_list:
            if 'idx' in list(e.keys()):

                if ' ' in e['idx']:
                    idx = list(map(int, e['idx'].split(' ')))
                else:
                    idx = ast.literal_eval(e['idx'])

                if type(idx) is int:

#                             assert tweet_split[idx] == e['text']
                    token_labels[idx] = 'B-'+e['className'].split('-')[1]
                else:
                    idx=list(idx)
                    token_labels[idx[0]] = 'B-'+e['className'].split('-')[1]
                    token_labels[idx[1:]] = 'I-' + e['className'].split('-')[1]

    return token_labels.tolist()

In [844]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
reviewsPertweet = defaultdict(list)
for d in parsed_3000:
    entity_labels=process_entity_label_3000(d)
    reviewsPertweet[d['InputId']].append([1]+[0]*(len(entity_labels)-1))


    for i in range(len(entity_labels)):
        user,item = d['WorkerId'], str(d['InputId'])+','+str(i)
        reviewsPerUser[user].append(d)
        reviewsPerItem[item].append({'WorkerId':d['WorkerId'],'entity_type':entity_labels[i]})

for d in parsed_200:
    entity_labels=process_entity_label_200(d)
    reviewsPertweet[d['InputId']].append([1]+[0]*(len(entity_labels)-1))


    for i in range(len(entity_labels)):
        user,item = d['WorkerId'], str(d['InputId'])+','+str(i)
        reviewsPerUser[user].append(d)
        reviewsPerItem[item].append({'WorkerId':d['WorkerId'],'entity_type':entity_labels[i]})

        
lu = len(reviewsPerUser)
li = len(reviewsPerItem)
data_ = np.empty((li, lu))
data_[:] = np.nan
# data__t = np.empty((lu, li))
ku = list(reviewsPerUser.keys())
ki = list(reviewsPerItem.keys())

# for i in range(li):
#     for r in reviewsPerItem[ki[i]]:
#         data_[i][ku.index(r['WorkerId'])] =r['entity_type']

# Construct the P Matrix
data_m = pd.DataFrame('',columns=ku,index=ki)
for i in range(li):
    for r in reviewsPerItem[ki[i]]:
        data_m.loc[ki[i]][ku.index(r['WorkerId'])] =r['entity_type']




In [845]:
list1=[i.split(',')[0] for i in list(data_m.index)]
all_tweet_id = []
 
for i in list1:
    if i not in all_tweet_id:
        all_tweet_id.append(i)
tokens_count=[list1.count(i) for i in all_tweet_id]

In [9]:
label2num={'O':8,'':-1,'B-food':2, 'B-loc':0, 'B-other':6, 'B-symptom':4,'I-food':3, 'I-loc':1,
       'I-other':7, 'I-symptom':5}

num2label={v:k for k, v in label2num.items()}

In [10]:
num2label

{8: 'O',
 -1: '',
 2: 'B-food',
 0: 'B-loc',
 6: 'B-other',
 4: 'B-symptom',
 3: 'I-food',
 1: 'I-loc',
 7: 'I-other',
 5: 'I-symptom'}

In [483]:
#annotations
new_data_m=data_m.replace(label2num)

In [484]:
## Create doc_start and features
ppdict = {n: grp.loc[n].to_dict('index') for n, grp in data_3000.set_index(['InputId', 'WorkerId']).groupby(level='Input.id')}

doc_start=[]
features=[]
for tweet_id,collection in ppdict.items():
    basic_info=list(collection.values())[0]
    txt = html.unescape(basic_info['Input.tweet'])
    temp = list(filter(None, re.split('([,.!?:()[\]"\s+])', txt)))

    # remove space strings from list and convert into np array
    tweet_split = list(filter(str.strip, temp))
    length=len(tweet_split)
    doc_start.extend([1]+[0]*(length-1))
    features.extend(tweet_split)

ppdict = {n: grp.loc[n].to_dict('index') for n, grp in data_200.set_index(['InputId', 'WorkerId']).groupby(level='Input.id')}

for tweet_id,collection in ppdict.items():
    basic_info=list(collection.values())[0]
    txt = html.unescape(basic_info['Input.tweet'])
    temp = list(filter(None, re.split('([,.!?:()[\]\\/"“”\s+])', txt)))

    # remove space strings from list and convert into np array
    tweet_split = list(filter(str.strip, temp))
    length=len(tweet_split)
    doc_start.extend([1]+[0]*(length-1))
    features.extend(tweet_split)

    
ll=list(reviewsPertweet.values())
doc_start=sum([i[0] for i in ll],[])
features=np.array(features)


In [485]:
from bayesian_combination import bayesian_combination

In [490]:
new_data_m.shape

(102159, 2517)

In [ ]:

num_classes = 9 # Beginning, Inside and Outside

bc_model = bayesian_combination.BC(L=num_classes, K=2515, annotator_model='seq', max_iter=20,tagging_scheme='IOB2',inside_labels=[1,3,5,7], outside_label=8, beginning_labels=[0,2,4,6])
values= bc_model.fit_predict(new_data_m.to_numpy(), np.array(doc_start), features)



In [ ]:
prob=values[0]

In [ ]:
agg=values[1]

In [ ]:
agg_predictions=[]
for i in tokens_count:
    agg_predictions.append(agg[:i].tolist())
    agg=np.delete(agg, range(i))

In [ ]:


final_agg=[list((pd.Series(agg_p)).map(num2label).values) for agg_p in agg_predictions]

In [ ]:
np.save('all_agreegation', np.array(final_agg), allow_pickle=True)

In [798]:
final_agg=np.load('all_agreegation.npy',allow_pickle=True)

In [947]:
all_3000=pd.read_json('all_processed.jaon')

In [948]:
id_3000=list(all_3000_sen.index)
idx_3000=[all_tweet_id.index(str(i)) for i in id_3000]
final_3000=[final_agg[i] for i in idx_3000]
all_3000_sen['bsc_entity_label']=final_3000

In [949]:
all_3000_sen['related_label']=all_3000_sen['related_label'].apply(ast.literal_eval)
all_3000_sen['entity_label']=all_3000_sen['entity_label'].apply(ast.literal_eval)
all_3000_sen['tweet_tokens']=all_3000_sen['tweet_tokens'].apply(ast.literal_eval)
all_3000_sen['sentence_score']=all_3000_sen['sentence_score'].apply(ast.literal_eval)



In [950]:
def mode(x): 
    try:
        return statistics.mode(x)
    except:
        return 1

In [951]:
def aggre_revel_label(all_3000_sen):
    all_new_record_label=[]
    for i in range(all_3000_sen.shape[0]):
        record=all_3000_sen.iloc[i]
        rl_match=[]
        for el in record['entity_label']:
            rl_match.append([1 if ae!='O' and ce==ae else np.nan for ce,ae in zip(el,record['bsc_entity_label'])])
        #     all_new_record_label.append(np.array(rl_match)*np.array(record['related_label']))
        mv_r=[]
        for x in (np.array(rl_match)*np.array(record['related_label'])).T:
            if np.nansum(x)>0:
                x=x[~np.isnan(x)]
                mv_r.append(mode(x))
            else: mv_r.append(np.nan)
        all_new_record_label.append(np.nan_to_num(np.array(mv_r)).tolist())
    return all_new_record_label 

In [952]:
all_3000_sen['new_related_label']=aggre_revel_label(all_3000_sen)

In [953]:
related_entity_label = []
for rl, el in zip(all_3000_sen['new_related_label'], all_3000_sen['bsc_entity_label']):
    new_el = [e if r ==1 else 'O' for r, e in zip(rl ,el)]
    # remove 'I-xxx' not after 'B-xxx' or 'I-xxx'
    for e, l in enumerate(new_el):
        if l.startswith('I-') and (e == 0 or new_el[e-1][2:] != l[2:]):
            new_el[e] = 'O'
    related_entity_label.append(new_el)
all_3000_sen['related_entity_label'] = related_entity_label

In [954]:
def remojize_word(s):
    s1 = re.sub(r'EMOJI_([0-9a-zA-Z\_]*)', r':\1:', s)
    return emoji.emojize(s1)

In [955]:
new_tokens_list, new_el_list, new_rl_list, new_rel_list = [], [], [], []
new_zip = zip(all_3000_sen['tweet_tokens'], all_3000_sen['bsc_entity_label'], all_3000_sen['new_related_label'], all_3000_sen['related_entity_label'])
for raw_tokens, raw_el, raw_rl, raw_rel in new_zip:
    emoji_pos = [p for p, i in enumerate(raw_tokens) if i.startswith('EMOJI_')]
    raw_tokens = [i if p not in emoji_pos else remojize_word(i) for p, i in enumerate(raw_tokens)]
    new_tokens = [i for p, i in enumerate(raw_tokens) if p-1 not in emoji_pos and p+1 not in emoji_pos]
    new_el = [i for p, i in enumerate(raw_el) if p-1 not in emoji_pos and p+1 not in emoji_pos]
    new_rl = [i for p, i in enumerate(raw_rl) if p-1 not in emoji_pos and p+1 not in emoji_pos]
    new_rel = [i for p, i in enumerate(raw_rel) if p-1 not in emoji_pos and p+1 not in emoji_pos]
    new_tokens_list.append(new_tokens)
    new_el_list.append(new_el)
    new_rl_list.append(new_rl)
    new_rel_list.append(new_rel)
all_3000_sen['new_tweet_tokens'] = new_tokens_list
all_3000_sen['n_entity_label'] = new_el_list
all_3000_sen['n_related_label'] = new_rl_list
all_3000_sen['n_related_entity_label'] = new_rel_list

In [1000]:
all_3000_sen['sc_me_1'] = all_3000_sen['sentence_score'].apply(lambda x: [int(int(i)>=3) for i in x])
all_3000_sen['msc_me_1'] = all_3000_sen['sc_me_1'].apply(mode)

In [795]:
check['entity_label_binary']=check['entity_label']
for i in range(check.shape[0]):
    check['entity_label_binary'].iloc[i]=[[0 if xi=='O' else 1 for xi in x] for x in check['entity_label'].iloc[i]]



In [868]:
check=all_3000_sen.loc[all_3000_sen['n_related_entity_label'].apply(lambda x: [float(i!='O') for i in x])!=all_3000_sen['n_related_label']]

## check sentence score and relevance enttiy

In [1108]:
## ignore the relevant entity of irrelevant tweets
def process_rl_in_irsen(re_label,sen_score):
    if sen_score==0 and sum(re_label)>0:
        output=[0]*len(re_label)
    else:
        output=re_label
    return output


all_aggregated_crowd['processed_n_related_label']=all_aggregated_crowd[['n_related_label','msc_me_1']].apply(lambda x: process_rl_in_irsen(x[0],x[1]),axis=1)



# all_aggregated_crowd['processed_n_related_label']=all_aggregated_crowd[['n_related_label','sen_mean_t']].apply(lambda x: process_rl_in_irsen(x[0],x[1]),axis=1)


## ignore the relevant entity of irrelevant tweets
def process_rle_in_irsen(re_label,sen_score,ree_label):
    if sen_score==0 and sum(re_label)>0:
        output=['O']*len(ree_label)
    else:
        output=ree_label
    return output


all_aggregated_crowd['processed_n_related_entity_label']=all_aggregated_crowd[['n_related_label','msc_me_1','n_related_entity_label']].apply(lambda x: process_rle_in_irsen(x[0],x[1],x[2]),axis=1)

# all_aggregated_crowd['processed_n_related_entity_label']=all_aggregated_crowd[['n_related_label','sen_mean_t','n_related_entity_label']].apply(lambda x: process_rle_in_irsen(x[0],x[1],x[2]),axis=1)

